In [1]:
import rasterio
import numpy as np
from pathlib import Path
import os
from scipy.ndimage import binary_dilation

In [2]:
# mask dilation for data augmentation
# 对1进行3个pixel的膨胀
def mask_dilation(mask_array, n=3):
    structure = np.ones((3, 3), dtype=np.bool_)
    dilated_mask = mask_array
    for _ in range(n):
        dilated_mask = binary_dilation(dilated_mask, structure=structure)
    return dilated_mask

In [13]:
indir = '/scratch2/ziyliu/pro_data/sate_dataset_V4/train'
tiles = [tile.name for tile in Path(indir).iterdir() if tile.is_dir()]

for tile in tiles:
    if not os.path.exists(indir + '/' + tile + '/mask_dilation3/'):
        os.makedirs(indir + '/' + tile + '/mask_dilation3/')
    for mask in os.listdir(indir + '/' + tile + '/mask/'):
        if mask.endswith(".tif"):
            mask_path = indir + '/' + tile + '/mask/' + mask
            with rasterio.open(mask_path) as src:
                mask_array = src.read(1)
                mask_profile = src.profile
                mask_array[mask_array > 0] = 1

                # 对1进行3个pixel的膨胀
                mask_array = mask_dilation(mask_array).astype(int)

                mask_out_path = indir + '/' + tile + '/mask_dilation3/' + mask
                with rasterio.open(mask_out_path, 'w', **mask_profile) as dst:
                    dst.write(mask_array, 1)